In [67]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import ast
from sklearn.decomposition import PCA
from sklearn import manifold
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

In [228]:
df = pd.read_csv('trb1.csv')

In [229]:
df = df.drop_duplicates()
df
# df = df[df['vdjdb.score'] != 0]

,cdr3,j.segm,mhc.a,mhc.b,mhc.class,antigen.epitope,vdjdb.score,v.seg,mhc.b_label,crd3_encode,...,j.segm_1,j.segm_2,j.segm_3,mhc.a_0,mhc.a_1,mhc.a_2,mhc.a_3,mhc.a_4,mhc.a_5,mhc.a_6
0,CATSDASLSSYNEQFF,TRBJ2-1*01,HLA-A*01,B2M,0,ATDALMTGY,0,TRBV24-1*01,0,"[0.49937695264816284, -0.6092822551727295, -0....",...,0.0,0.0,0.0,0,0,0,0,0,0,1
1,CASSLRTAGYNEQFF,TRBJ2-1*01,HLA-A*01,B2M,0,ATDALMTGY,0,TRBV27*01,0,"[-0.534170925617218, 0.2944409251213074, 0.852...",...,0.0,0.0,0.0,0,0,0,0,0,0,1
2,CASSLWGADGYTF,TRBJ1-2*01,HLA-A*01,B2M,0,ATDALMTGY,0,TRBV5-1*01,0,"[-0.534170925617218, 0.2944409251213074, 0.852...",...,0.0,1.0,0.0,0,0,0,0,0,0,1
3,CATSSAAQETQYF,TRBJ2-5*01,HLA-A*01,B2M,0,TDLGQNLLY,0,TRBV15*01,0,"[0.49937695264816284, -0.6092822551727295, -0....",...,1.0,0.0,0.0,0,0,0,0,0,0,1
4,CASRSLLGTRVETEAFF,TRBJ1-1*01,HLA-A*01,B2M,0,ATDALMTGY,0,TRBV6-1*01,0,"[-2.708209753036499, 0.4491012692451477, -0.38...",...,0.0,0.0,1.0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49680,CASSYSIRGSRGEQFF,TRBJ2-1*01,HLA-E*01:03,B2M,0,RLPAKAPL,3,TRBV6-5*01,0,"[-0.534170925617218, 0.2944409251213074, 0.852...",...,0.0,0.0,0.0,1,1,0,1,1,1,0
49682,CASSYSIRGSRGEQFF,TRBJ2-1*01,HLA-E*01:03,B2M,0,RLPAKAPLLGCG,3,TRBV6-5*01,0,"[-0.534170925617218, 0.2944409251213074, 0.852...",...,0.0,0.0,0.0,1,1,0,1,1,1,0
49683,CASSLGREYGYTF,TRBJ1-2*01,HLA-E*01:03,B2M,0,RMYSPVSI,3,TRBV7-9*01,0,"[-0.534170925617218, 0.2944409251213074, 0.852...",...,0.0,1.0,0.0,1,1,0,1,1,1,0
49684,CASSLGREYGYTF,TRBJ1-2*01,HLA-E*01:03,B2M,0,RMYSPVSIL,3,TRBV7-9*01,0,"[-0.534170925617218, 0.2944409251213074, 0.852...",...,0.0,1.0,0.0,1,1,0,1,1,1,0


In [230]:
value_counts = df['antigen.epitope'].value_counts(normalize=True)

threshold = 0.01
df = df[df['antigen.epitope'].isin(value_counts[value_counts > threshold].index)]

In [231]:
grouped_df = df.groupby(['cdr3','crd3_encode', 'j.segm', 'v.seg', 'mhc.a', 'mhc.b',
                         'v.seg_0', 'v.seg_1', 'v.seg_2', 'v.seg_3', 'v.seg_4', 'v.seg_5',
                         'j.segm_0', 'j.segm_1', 'j.segm_2', 'j.segm_3',
                         'mhc.a_0', 'mhc.a_1', 'mhc.a_2', 'mhc.a_3', 'mhc.a_4', 'mhc.a_5', 'mhc.a_6', 'mhc.b_label', 'mhc.class', 'antigen.epitope']).max()['vdjdb.score']
df = grouped_df.reset_index()
df = df.drop_duplicates()
df.to_csv('prediction.csv', index=False)

In [2]:
# df = pd.read_csv('111.csv')

# mask =(df.groupby('antigen_label')['antigen_label'].transform('count') == 1)

In [232]:
# df = df[~mask]
encoder = LabelEncoder()
label_rencoded = encoder.fit_transform(df['antigen.epitope'].tolist())
df['label_rencoded'] = label_rencoded
df

,cdr3,crd3_encode,j.segm,v.seg,mhc.a,mhc.b,v.seg_0,v.seg_1,v.seg_2,v.seg_3,...,mhc.a_2,mhc.a_3,mhc.a_4,mhc.a_5,mhc.a_6,mhc.b_label,mhc.class,antigen.epitope,vdjdb.score,label_rencoded
0,CAAAERNTGELFF,"[-0.16498591005802155, -0.1462753862142563, 0....",TRBJ2-2*01,TRBV28*01,HLA-A*02,B2M,0.0,1.0,1.0,0.0,...,0,0,1,0,0,0,0,YLQPRTFLL,0,12
1,CAAEDPEWGAEAFF,"[-0.24984240531921387, 0.38962453603744507, -0...",TRBJ1-1*01,TRBV30*01,HLA-A*03:01,B2M,0.0,1.0,1.0,0.0,...,0,1,1,1,1,0,0,KLGGALQAK,0,6
2,CAAGAGLSYEQYF,"[-0.7613160014152527, 0.263674259185791, -0.52...",TRBJ2-7*01,TRBV27*01,HLA-A*02,B2M,0.0,1.0,0.0,1.0,...,0,0,1,0,0,0,0,NLVPMVATV,0,8
3,CAAGDANTGELFF,"[-0.7613160014152527, 0.263674259185791, -0.52...",TRBJ2-2*01,TRBV5-1*01,HLA-A*02:01,B2M,0.0,1.0,1.0,1.0,...,0,0,1,0,1,0,0,YLQPRTFLL,0,12
4,CAAGEMFGLGETQYF,"[-0.7613160014152527, 0.263674259185791, -0.52...",TRBJ2-5*01,TRBV4-2*01,HLA-A*03:01,B2M,0.0,1.0,1.0,1.0,...,0,1,1,1,1,0,0,KLGGALQAK,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29760,CVTGQGDINEQFF,"[0.07739217579364777, 0.0676565170288086, 0.14...",TRBJ2-1*01,TRBV11-1*01,HLA-A*03:01,B2M,0.0,0.0,0.0,1.0,...,0,1,1,1,1,0,0,KLGGALQAK,0,6
29761,CVTSAPLRQGLFNYGYTF,"[0.17337776720523834, 0.1124500036239624, 0.03...",TRBJ1-2*01,TRBV24-1*01,HLA-A*02,B2M,0.0,1.0,0.0,1.0,...,0,0,1,0,0,0,0,GILGFVFTL,0,3
29762,CWGNTEAFF,"[-0.13605011999607086, 0.22847767174243927, 0....",TRBJ1-1*01,TRBV12-4*01,HLA-A*03:01,B2M,0.0,0.0,1.0,0.0,...,0,1,1,1,1,0,0,KLGGALQAK,0,6
29763,IPDTGELFF,"[0.6372780799865723, 0.36321160197257996, 0.24...",TRBJ2-2*01,TRBV3-1*01,HLA-DPA*01:03,HLA-DPB*04:01,0.0,1.0,1.0,0.0,...,1,0,0,1,0,1,1,TFEYVSQPFLMDLE,0,11


In [233]:
X = df[['crd3_encode', 'v.seg_0', 'v.seg_1', 'v.seg_2', 'v.seg_3', 'v.seg_4', 'v.seg_5','vdjdb.score',
              'j.segm_0', 'j.segm_1', 'j.segm_2', 'j.segm_3',
              'mhc.a_0', 'mhc.a_1', 'mhc.a_2', 'mhc.a_3', 'mhc.a_4', 'mhc.a_5', 'mhc.a_6', 'mhc.b_label', 'mhc.class']].fillna(0)
y = df['label_rencoded']

In [234]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, stratify=y_train)

In [44]:
y_train

array([ 2,  4, 19, ...,  9, 17, 18], dtype=int64)

In [106]:
X_train

,crd3_encode,v.seg_0,v.seg_1,v.seg_2,v.seg_3,v.seg_4,v.seg_5,vdjdb.score,j.segm_0,j.segm_1,...,j.segm_3,mhc.a_0,mhc.a_1,mhc.a_2,mhc.a_3,mhc.a_4,mhc.a_5,mhc.a_6,mhc.b_label,mhc.class
37507,"[-0.534170925617218, 0.2944409251213074, 0.852...",0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0,0
22069,"[-0.534170925617218, 0.2944409251213074, 0.852...",0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0,0
22689,"[-0.534170925617218, 0.2944409251213074, 0.852...",1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0,0
30365,"[-0.534170925617218, 0.2944409251213074, 0.852...",1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0,0
6654,"[-0.534170925617218, 0.2944409251213074, 0.852...",1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6176,"[0.04405900090932846, 0.28644683957099915, 0.1...",0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0
26121,"[-2.708209753036499, 0.4491012692451477, -0.38...",0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0,0
14767,"[-0.534170925617218, 0.2944409251213074, 0.852...",1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0,0
9671,"[-0.534170925617218, 0.2944409251213074, 0.852...",0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0


In [235]:
X_train_copy_list = []
y_train_copy_list = []

# 遍历 X_train 和 y_train 中的每一行
for index, row in X_train.iterrows():
    # 根据 score 列的值确定要复制的数量
    num_copies = row['vdjdb.score']
    # 复制当前行，并添加到列表中
    if num_copies > 0:
        X_train_copy_list.extend([row] * num_copies)
        y_train_copy_list.extend([y_train[index]] * num_copies)

# 将列表中的数据片段合并为新的 DataFrame
X_train_copy = pd.concat(X_train_copy_list, axis=1).T
y_train_copy = pd.Series(y_train_copy_list, name='label_rencoded')

X_train_copy.reset_index(drop=True, inplace=True)
y_train_copy.reset_index(drop=True, inplace=True)

X_train_new = pd.concat([X_train, X_train_copy], ignore_index=True)
y_train_new = pd.concat([y_train, y_train_copy], ignore_index=True)

# 重新设置索引
X_train_new.reset_index(drop=True, inplace=True)
y_train_new.reset_index(drop=True, inplace=True)

In [150]:
y_train_new.nunique

<bound method IndexOpsMixin.nunique of 0        6
1        6
2        3
3        6
4        8
        ..
26009    1
26010    8
26011    8
26012    4
26013    3
Name: label_rencoded, Length: 26014, dtype: int64>

In [236]:
def dimensionality_reduction(df, n_components1, n_components2):
    vdj = df[['v.seg_0', 'v.seg_1', 'v.seg_2', 'v.seg_3', 'v.seg_4', 'v.seg_5',
              'j.segm_0', 'j.segm_1', 'j.segm_2', 'j.segm_3']]
    mhc = df[['mhc.a_0', 'mhc.a_1', 'mhc.a_2', 'mhc.a_3', 'mhc.a_4', 'mhc.a_5', 'mhc.a_6', 'mhc.b_label']]

    vdj_lists = vdj.values.tolist()
    mhc_lists = mhc.values.tolist()
    
    cdr3_kmer = list(df['crd3_encode'].apply(lambda x: ast.literal_eval(x)))
    
    
    pca = PCA(n_components = n_components1)
    pca_data = pca.fit_transform(cdr3_kmer)
    
    tcr_list = []
    for cdr3, vdj, mhc in zip(pca_data, vdj_lists, mhc_lists):
        tcr_list.append(list(cdr3) + vdj + mhc)
    
    tsne = manifold.TSNE(n_components=n_components2, init='pca', random_state=1, learning_rate=100)

    reduced_data = tsne.fit_transform(tcr_list)
    return pca_data, reduced_data

In [237]:
train_pca, X_train_preprocessing = dimensionality_reduction(X_train_new, 50, 3)
test_pca, X_test_preprocessing = dimensionality_reduction(X_test, 50, 3)

In [238]:
train_pca


array([[ -8.03680179,   5.52354122,  -0.13524491, ...,  -1.11256149,
          0.75924602,   1.91240422],
       [-11.48089868,  -2.32729078,  -1.60201434, ...,  -0.09162392,
         -0.99544878,  -0.17460618],
       [ -7.93063969,  -3.20835042,  -0.72638646, ...,  -1.63338548,
          0.041282  ,   1.42595615],
       ...,
       [ -6.05081283,  -4.46358837,   8.3196395 , ...,  -1.15303466,
          1.02922587,   1.00330027],
       [ -9.24992686,  -1.86530262,  -1.40058823, ...,   0.28539827,
         -1.20275641,   2.85894463],
       [  5.35372347,   0.07537102,  -5.02169088, ...,  -1.12174791,
         -0.26682216,  -0.32202362]])

In [154]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_new), y=y_train_new)
class_weights

array([1.49893402, 1.44691028, 4.22168127, 0.61269961, 1.83416767,
       4.24857096, 0.19626098, 3.93138885, 0.4347332 , 2.18936206,
       4.41738835, 4.57912339, 2.2687947 ])

In [59]:
np.unique(y_train)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20], dtype=int64)

In [156]:
def custom_scorer(y_true, y_pred):
    auc_scores = []
    for i in range (len(np.unique(y_true))):
        y_pred_class = y_pred[:, i]
        y_true_class = (y_true == i)
        auc_scores.append(roc_auc_score(y_true_class, y_pred_class))
    auc_score = sum(auc_scores) / len(np.unique(y_train))
    return auc_score

custom_scorer = make_scorer(custom_scorer)

In [10]:
# Define the parameter grid
param_grid = {
    'n_estimators': [80, 100, 150, 200],
    'learning_rate': [0.02, 0.05, 0.1],
    'max_depth': [6, 8, 10, 12],
    'min_child_weight': [2, 3, 4, 5, 6],
    'gamma': [0, 0.1, 0.2, 0.3],
    'reg_alpha' : [0, 0.05, 0.1, 1]
}

# Create an XGBoost classifier
xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(np.unique(y_train)), random_state = 42, nthread = 2, eval_metric = 'mlogloss', scale_pos_weight=class_weights)

# Perform grid search
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose = 1, scoring='roc_auc_ovr')
grid_search.fit(X_train_preprocessing, y_train, sample_weight=X_train['vdjdb.score'].values)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Fitting 5 folds for each of 15360 candidates, totalling 76800 fits
Best parameters:  {'gamma': 0.1, 'learning_rate': 0.05, 'max_depth': 12, 'min_child_weight': 2, 'n_estimators': 150, 'reg_alpha': 0.05, 'subsample': 0.7}
Best score:  0.8055113898679627


In [239]:
# Create an instance of XGBClassifier
xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class= len(np.unique(y_train)), eval_metric = 'mlogloss',
          learning_rate=0.05, n_estimators= 150, max_depth= 12, min_child_weight= 2, seed=0, gamma= 0.1, reg_alpha = 0.05, subsample = 0.7)


In [240]:
xgb_model.fit(train_pca, y_train_new)
y_pca_pred = xgb_model.predict_proba(test_pca)

In [241]:
# 计算每个类别的AUC
auc_scores = []
for i in range (len(np.unique(y_test))):
    y_pred_class = y_pca_pred[:, i]  # 提取第i个类别的预测概率
    y_true_class = (y_test == i)  # 将真实标签转化为二分类问题
    auc_scores.append(roc_auc_score(y_true_class, y_pred_class))

# 计算多分类任务的AUC指标
mean_auc = sum(auc_scores) / len(np.unique(y_train))

print('AUC: ', mean_auc)

AUC:  0.5802965955803523


In [242]:
xgb_model.fit(X_train_preprocessing, y_train_new)
pre_y_test = xgb_model.predict_proba(X_test_preprocessing)

In [243]:
# 计算每个类别的AUC
auc_scores = []
for i in range (len(np.unique(y_test))):
    y_pred_class = pre_y_test[:, i]  # 提取第i个类别的预测概率
    y_true_class = (y_test == i)  # 将真实标签转化为二分类问题
    auc_scores.append(roc_auc_score(y_true_class, y_pred_class))

# 计算多分类任务的AUC指标
mean_auc = sum(auc_scores) / len(np.unique(y_train))

print('AUC: ', mean_auc)

AUC:  0.5995250231219331
